In [1]:
from astropy.io import fits
import numpy as np
from core.distortion import d2cMapping
from os import listdir
from core.funcs import point_source_centroiding
import glob as glob
from os import mkdir
from os.path import isdir


In [2]:
%matplotlib notebook

In [ ]:
##Change
dataDir = "../DATA/twhya/sci_interp_bfe/"
refdir = './core/FRINGE/'
outdir = '../DATA/twhya/sci_bfe_fringe/'

file_list = [aa.split("/")[-1] for aa in glob.glob(dataDir+"/*_rate.fits")]
version = '01.00.00'
if not isdir(outdir): mkdir(outdir) 


In [4]:
def return_centroids(rate,b,d2cMaps):
    
    if b[1:] == 'SHORT':
        subband = 'A'
    elif b[1:] == 'MEDIUM':
        subband = 'B'
    else:
        subband = 'C'
    
    lambmin = np.nanmin(d2cMaps['lambdaMap'][np.where(d2cMaps['lambdaMap']!=0)]) # micron
    lambmax = np.nanmax(d2cMaps['lambdaMap']) # micron
    lambcens = np.arange(lambmin,lambmax,(lambmax-lambmin)/1024.)
    lambfwhms = np.ones(len(lambcens))*(2*(lambmax-lambmin)/1024.)
    
    sign_amp2D,alpha_centers2D,beta_centers2D,sigma_alpha2D,sigma_beta2D,bkg_amp2D = point_source_centroiding(b,rate,d2cMaps,spec_grid=[lambcens,lambfwhms],fit='2D')

    alpha_ind=np.where(np.abs(alpha_centers2D)<=np.nanmax(np.abs(d2cMaps['alphaMap'])))[0][:]
    beta_ind=np.where(np.abs(beta_centers2D)<=np.nanmax(np.abs(d2cMaps['betaMap'])))[0][:]

    popt_alpha = np.polyfit(lambcens[alpha_ind],alpha_centers2D[alpha_ind],4)
    alpha = np.poly1d(popt_alpha)

    popt_beta  = np.polyfit(lambcens[beta_ind],beta_centers2D[beta_ind],4)
    beta = np.poly1d(popt_beta)

    return alpha(lambcens[512]),beta(lambcens[512])

def find_nearest_grid(cent_alpha,cent_beta,band,fringedir,reflist):

    diff = 1000
    
    diffalpha = []
    diffbeta = []
    filelist = []
    
    for filename in reflist:
        hdu = fits.open(fringedir+filename)
        refalpha = hdu[1].header['A{}'.format(band[0])]
        refbeta = hdu[1].header['B{}'.format(band[0])]
        hdu.close()

        diffalpha.append(refalpha-cent_alpha)
        diffbeta.append(refbeta-cent_beta)
        filelist.append(filename)

    lowest = sorted(np.abs(diffbeta))[:3]

    for i in range(len(diffbeta)):
        if np.abs(diffbeta[i]) in lowest:
            if np.abs(diffalpha[i]) < diff:
#                 print(diff,diffalpha[i])
                reffile = filelist[i]
                diff = np.abs(diffalpha[i])
#                 print(diff)
    
    return reffile

In [5]:
file_list

['jw01282020001_03106_00004_mirifushort_rate.fits',
 'jw01282020001_03106_00003_mirifulong_rate.fits',
 'jw01282019001_02103_00001_mirifushort_rate.fits',
 'jw01282020001_03102_00001_mirifulong_rate.fits',
 'jw01282020001_03102_00004_mirifulong_rate.fits',
 'jw01282020001_03102_00002_mirifushort_rate.fits',
 'jw01282020001_03102_00004_mirifushort_rate.fits',
 'jw01282020001_03104_00003_mirifushort_rate.fits',
 'jw01282019001_02101_00001_mirifulong_rate.fits',
 'jw01282020001_03106_00002_mirifulong_rate.fits',
 'jw01282020001_03104_00004_mirifulong_rate.fits',
 'jw01282020001_03104_00001_mirifushort_rate.fits',
 'jw01282020001_03104_00001_mirifulong_rate.fits',
 'jw01282019001_02102_00001_mirifushort_rate.fits',
 'jw01282020001_03106_00002_mirifushort_rate.fits',
 'jw01282020001_03102_00001_mirifushort_rate.fits',
 'jw01282019001_02101_00001_mirifushort_rate.fits',
 'jw01282020001_03104_00002_mirifulong_rate.fits',
 'jw01282019001_02103_00001_mirifulong_rate.fits',
 'jw01282020001_03102

In [7]:
for scifile in file_list:
    hdu = fits.open(dataDir + scifile)

    detector = hdu[0].header['DETECTOR']
    subbandl = hdu[0].header['BAND']
    
    try : 
        dithdir = hdu[0].header['DITHDIRC']
    except :
       pass
   
    if subbandl == 'SHORT':
        subband = 'A'
    elif subbandl == 'MEDIUM':
        subband = 'B'
    else:
        subband = 'C'
        
    if detector == 'MIRIFUSHORT':
        band = ['1{}'.format(subband),'2{}'.format(subband)]
    else:
        band = ['3{}'.format(subband),'4{}'.format(subband)]

    data = hdu['SCI'].data

    hdu.close()

    fringe_file = {}

    if detector == 'MIRIFUSHORT':
        alpha = {}
        beta = {}
        for b in band:
            
            reffilelist = []
            for filename in listdir('./core/FRINGE/'):
                if filename.startswith('MIRIFUSHORT_{}'.format(subbandl)) and filename.endswith('{}.fits'.format(version)):
                    reffilelist.append(filename)
            
            d2cMaps = d2cMapping(b,'./data/',slice_transmission='10pc',fileversion = "flt7")
            alpha[b],beta[b] = return_centroids(data,b,d2cMaps)
            
            fringe_file[b] = find_nearest_grid(alpha[b],beta[b],b,refdir,reffilelist)
            print(fringe_file[b])
    else:
        num = (scifile.split('_')[2])[-1]
        for b in band:
            fringe_file[b] = 'MIRIFULONG_{}_PS_FRINGE_{}{}_{}.fits'.format(subbandl,dithdir,num,version)


    if detector == 'MIRIFUSHORT':
        for b in band:
            fringe_flat = fits.getdata(refdir + fringe_file[b])
            
            if b[0]=='1':
                data[:,:512] = data[:,:512]/fringe_flat[:,:512]
            else:
                data[:,512:] = data[:,512:]/fringe_flat[:,512:]
    else:
        for b in band:
            fringe_flat = fits.getdata(refdir + fringe_file[b])
            data = data/fringe_flat
        
    hdu = fits.open(dataDir + scifile)
    hdu['SCI'].data = data

    hdu.writeto(outdir + scifile,overwrite=True)

    hdu.close()

STEP 1: Rough centroiding
Slice 9 has the largest summed flux
Source position: beta = -0.35arcsec, alpha = -0.17arcsec 

STEP 2: 1D Gaussian fit
[Along-slice fit] The following bins failed to converge:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023]
[Across-slice fit] The following bins failed to converge:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023]

STEP 3: 2D Gaussian fit
The following bins failed to converge:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 

/opt/miniconda3/envs/minds_env/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


[Along-slice fit] The following bins failed to converge:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 1019, 1020, 1021, 1022, 1023]
[Across-slice fit] The following bins failed to converge:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023]

STEP 3: 2D Gaussian fit
The following bins failed to converge:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46